# Overview

The purpose of this notebook is to evaluate the original and the current implementation of misorientation calculations.

In [1]:
import numpy as np

from hexomap.past   import Misorien2FZ1 as miso_new
from hexomap.past   import EulerZXZ2Mat as euler2mat_new
from hexomap.past   import generate_random_rot_mat

from hexomap.RotRep import Misorien2FZ1 as miso_org   # one symop
from hexomap.RotRep import Misorien2FZ2 as miso_org2  # two symops
from hexomap.RotRep import EulerZXZ2Mat as euler2mat_old

from hexomap.orientation import sym_operator
from hexomap.orientation import Quaternion
from hexomap.orientation import Orientation
from hexomap.orientation import Eulers
from hexomap.orientation import Frame

In [2]:
miso_new??

Signature: miso_new(m1, m2, symtype='Cubic')
Docstring: <no docstring>
Source:   
def Misorien2FZ1(m1, m2, symtype='Cubic'):
    dqcore = Quaternion.from_matrix(m1.T@m2)
    dqs  = [dqcore*op for op in sym_operator(symtype)]
    angs = [q.rot_angle for q in dqs]
    idx  = np.argmin(angs)
    return dqs[idx].as_matrix, angs[idx]
File:      ~/lib/hexomap/past.py
Type:      function


In [3]:
miso_org??

Signature: miso_org(m1, m2, symtype='Cubic')
Source:   
def Misorien2FZ1(m1, m2, symtype='Cubic'):
    """
    Careful, it is m1*op*m2T, the misorientation in sample frame, the order matters. Only returns the angle, doesn't calculate the right axis direction

    Parameters
    -----------
    m1:     ndarray
            Matrix representation of orientation1
    m2:     ndarray
            Matrix representation of orientation2
    symtype:string
            The crystal symmetry

    Returns
    -----------
    oRes:   ndarray
            The misorientation matrix after reduced. Note that this function doesn't actually
            reduce the orientation to fundamental zone, only make sure the angle is the
            smallest one, so there are multiple orientations have the same angle but
            different directions. oRes is only one of them.
    angle:  scalar
            The misorientation angle.
    """
    #m2 = np.matrix(m2)
    ops = GetSymRotMat(symtype)
    angle = 6.3
    


# Test_1: Consistency

The first test evaluate the consistence bewteen the original and the new implementation of two misorientation calculation.

```python
def quaternions_from_eulers(eulers: np.ndarray) -> np.ndarray:
    """ Return a quaternion based on given Euler Angles """
    # allow euler as an numpy array
    try:
        eulers = eulers.reshape((-1, 3))
    except:
        raise ValueError(f"Eulers angles much be ROW/horizontal stacked")

    ee = 0.5*eulers
    cs = np.cos(ee)
    ss = np.sin(ee)
    c1, c, c2 = cs[:,0], cs[:,1], cs[:,2]
    s1, s, s2 = ss[:,0], ss[:,1], ss[:,2]
    
    quats = np.empty([eulers.shape[0], 4])
    quats[:,0] =  c1*c*c2 - s1*c*s2
    quats[:,1] =  c1*s*c2 + s1*s*s2
    quats[:,2] = -c1*s*s2 + s1*s*c2
    quats[:,3] =  c1*c*s2 + s1*c*c2

    return (quats/np.linalg.norm(quats, axis=1)[:,None]) * np.sign(quats[:,0])[:,None]
```

In [4]:
euler = (np.random.random(3)-0.5)*4*np.pi

q = Quaternion.from_eulers(euler)
print(f"{euler} -> {q.as_array}")

print()
eulers = np.vstack((euler, euler))
qs = Quaternion.quaternions_from_eulers(eulers)
print(eulers)
print("|")
print("v")
print(qs)

[-2.18079375  0.57518029 -2.0115243 ] -> [ 0.48092922 -0.2826269   0.02397733  0.82961088]

[[-2.18079375  0.57518029 -2.0115243 ]
 [-2.18079375  0.57518029 -2.0115243 ]]
|
v
[[ 0.48092922 -0.2826269   0.02397733  0.82961088]
 [ 0.48092922 -0.2826269   0.02397733  0.82961088]]


In [5]:
def check(m1, m2):
    for lb, me in {
    "current": miso_new, 
    "org1":    miso_org, 
    # "org2":    miso_org2
    }.items():
        print(f"{lb}\t{np.degrees(me(m1, m2)[1])}")

In [6]:
m1 = euler2mat_new(np.radians([0,  45,  0]))
m2 = euler2mat_new(np.radians([98, 45,  0]))
check(m1, m2)

m1 = euler2mat_new(np.radians([0,  45,  0]))
m2 = euler2mat_new(np.radians([0,  45, 98]))
check(m1, m2)

current	60.98631277324766
org1	60.98631277324768
current	7.9999999999999
org1	7.999999999999981


In [7]:
for _ in range(10):
    m1 = euler2mat_new((np.random.random(3)-0.5)*5*np.pi)
    m2 = euler2mat_new((np.random.random(3)-0.5)*5*np.pi)
    
    check(m1, m2)

current	37.28316471355922
org1	37.283164713559245
current	48.30848916372926
org1	48.30848916372931
current	39.85150205126839
org1	39.85150205126844
current	50.138298864832976
org1	50.13829886483299
current	13.730973075076875
org1	13.730973075076887
current	32.44288075881981
org1	32.44288075881982
current	54.42141450854485
org1	54.421414508544885
current	45.879521056341154
org1	45.879521056341154
current	33.74294831332352
org1	33.74294831332352
current	46.98172944935342
org1	46.98172944935346


# Test_2 speed

In [17]:
m1 = euler2mat_new((np.random.random(3)-0.5)*2*np.pi)
m2 = euler2mat_new((np.random.random(3)-0.5)*2*np.pi)

In [30]:
%time miso_new(m1, m2)

CPU times: user 3.45 ms, sys: 808 µs, total: 4.26 ms
Wall time: 3.56 ms


(array([[ 0.9057088 , -0.31486668,  0.28381427],
        [ 0.18512355,  0.89610799,  0.40338535],
        [-0.38134084, -0.31280895,  0.86990213]]), 0.5810997368069682)

In [31]:
%time miso_org(m1, m2)

CPU times: user 1.1 ms, sys: 1.22 ms, total: 2.32 ms
Wall time: 1.22 ms


(array([[ 0.83695182, -0.47294059,  0.27538854],
        [ 0.45895106,  0.8806494 ,  0.11756087],
        [-0.29812006,  0.02799708,  0.9541177 ]]), 0.5810997368069685)

Due to the instance initialization, there is still some overhead here...